# Analyze histogram of predicted scores

In [ ]:
import sys
# sys.path.append("./../")
sys.path.insert(0, "./../")

In [ ]:
import os
import json
import glob

import numpy as np
from scipy import stats as S

import torch
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE

from tbparse import SummaryReader

import methods, models, datasets, transforms

import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
def get_event_file(model_dir):
    tblog_dir = os.path.join(model_dir, "tblog")
    events_file = glob.glob(tblog_dir + "/events.out.tfevents*.1")[0] # *.1 will be from test

    return events_file

In [ ]:
model_dirs = [
    "./../zoo/additional-models/fig-1/BinaryMNISTC-1000-53-identity/LeNet/mfvi-sz1000-5-20220827070550",
    "./../zoo/additional-models/fig-1/BinaryMNISTC-1000-53-identity/LeNet/sl-alpha1e+04-beta-1e-01-1e-01-20221012113746/",
    "./../zoo/additional-models/fig-1/BinaryMNISTC-1000-53-identity/LeNet/sl-alpha1e+04-beta-1e+00-1e+00-20221012113746",
    "./../zoo/additional-models/fig-1/BinaryMNISTC-1000-53-identity/LeNet/sl-alpha1e+04-beta-5e+00-5e+00-20221012113746"
]

n = len(model_dirs)

In [ ]:
fig, ax = plt.subplots(1, n, figsize=(6*n, 3))

for _ax, model_dir in zip(ax, model_dirs):
    config_json = os.path.join(model_dir, "config.json")
    event_file = get_event_file(model_dir)
    config = json.load(open(config_json, 'r'))
    params = config['method_params']

    ev = SummaryReader(event_file, pivot=True)
    df_scalars = ev.scalars
    df_hist = ev.histograms

    l = df_hist.iloc[-1]
    x = l['y_pred_test/limits']
    y = l['y_pred_test/counts']
    y = y / y.sum()

    # base_measure = S.beta(params['a'], params['b'])
    # x2 = np.linspace(0.0, 1.0, 25)
    # pdf = base_measure.pdf(x2)

    _ax.bar(x, y, width=-0.09, align='edge')
    # plt.plot(x2, pdf)
    _ax.set_xlim(0.0, 1.0)
    _ax.set_ylim(0.0, 0.6)
    # plt.title(
    #     "Beta({:.2f}, {:.2f})".format(params['a'], params['b']) +
    #     "    $\\alpha$ = {:.0f}".format(params['alpha']) +
    #     "    $\\lambda_{SL}$ = "+"{:.2f}".format(params['lam_sl'])
    # )
    _ax.set_xlabel("$\\hat{p}$")
    _ax.set_ylabel("$\hat{s_{\\theta}}$")

## Plot decision regions

Build TSNE embeddings of samples

In [ ]:
DatasetClass = getattr(datasets, config['dataset']) # Will automatically detect the dataset for testing
TransformClass = getattr(transforms, config['transform'])
testset = DatasetClass(**config['ds_params'], split='test', transform=TransformClass())
K = testset.n_labels

batch_size = 1000
dataloader = DataLoader(testset, batch_size=batch_size, shuffle=False)
batch = next(iter(dataloader))
sample_images, sample_target = batch
X = sample_images.reshape(batch_size,-1).numpy()
X_embedded = TSNE(n_components=2, init='random').fit_transform(X)
x = X_embedded[:,0]; y = X_embedded[:,1]

#### Get predictions from the model

In [ ]:
def get_predictions(model, dataset, N=10):
    dataloader = DataLoader(dataset, batch_size=1024, shuffle=False)
    torch.manual_seed(42)

    # Get predictions from model
    ypreds = []
    with torch.no_grad():
        for x, y in dataloader:
            _ypreds, _ = model.sample_predictions(x, n=N)
            if ypreds:
                for i in range(N):
                    ypreds[i] = torch.cat((ypreds[i], _ypreds[i]), dim=0)
            else:
                for y2 in _ypreds:
                    # y2 is a tensor of shape (B, K)
                    ypreds.append(y2)

    # Convert to softmax score from log_softmax
    yprobs = [torch.exp(_ypreds) for _ypreds in ypreds]
    
    return yprobs

In [ ]:
z = []

for model_dir  in model_dirs:
    config_json = os.path.join(model_dir, "config.json")
    ckpt_file = glob.glob(model_dir + '/step=*.ckpt')[-1]

    config = json.load(open(config_json, 'r'))

    MethodClass = getattr(methods, config['method'])
    DatasetClass = getattr(datasets, config['dataset']) # Will automatically detect the dataset for testing
    ModelClass = getattr(models, config['model'])
    TransformClass = getattr(transforms, config['transform'])

    testset = DatasetClass(**config['ds_params'], split='test', transform=TransformClass())
    K = testset.n_labels

    model = MethodClass.load_from_checkpoint(ckpt_file, model=ModelClass(K), strict=False)
    torch.manual_seed(24)
    _ypreds, _ = model.sample_predictions(sample_images, n=1)
    yprobs = torch.exp(_ypreds[0].detach())
    _z = yprobs[:,1].numpy()

    z.append(_z)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = n, figsize = (5*n, 3.4))

xlim = (-25,25); ylim = (-25,25)
levels = 15; ms = 1

for _ax, _z in zip(ax, z):
    # _ax.tricontour(x, y, _z, levels=levels, linewidths=0.5, colors='k')
    cntr = _ax.tricontourf(x, y, _z, levels=levels, cmap="RdBu_r", vmin=0.0, vmax=1.0) 
    fig.colorbar(cntr, ax=_ax)
    _ax.set(xlim=xlim, ylim=ylim)
    _ax.plot(x, y, 'ko', ms=ms)
    _ax.set_axis_off()